In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.applications import Xception, MobileNetV3Large
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

# Ruta y carga de datos
ruta_base = '/content/drive/MyDrive/SCUBI DU'
def is_valid_image(file_path):
    valid_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']
    return any(file_path.lower().endswith(ext) for ext in valid_extensions)

clases_unicas = sorted([d for d in os.listdir(ruta_base) if os.path.isdir(os.path.join(ruta_base, d))])
class_to_label = {name: idx for idx, name in enumerate(clases_unicas)}

imagenes_cargadas, etiquetas_cargadas = [], []
for class_name in clases_unicas:
    class_folder = os.path.join(ruta_base, class_name)
    valid_images = [os.path.join(class_folder, f) for f in os.listdir(class_folder) if is_valid_image(f)]
    for image_path in valid_images:
        img = load_img(image_path, target_size=(224, 224))
        imagenes_cargadas.append(np.array(img))
        etiquetas_cargadas.append(class_to_label[class_name])

X = np.array(imagenes_cargadas) / 255.0
y = to_categorical(np.array(etiquetas_cargadas))

# Dividir en entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

# Parámetros óptimos fijos
parametros_fijos = {
    'batch_size': 6,
    'activation': 'sigmoid',
    'patience': 3,
    'epochs': 10,
    'learning_rate': 0.0001
}

# Configuraciones de capas
layer_unit_configs = {
    1: [[64], [128], [256]],
    2: [[256, 128], [128, 64], [64, 32]],
    3: [[256, 128, 64], [128, 64, 32], [64, 32, 16]]
}

# Valores de tolerancia
modelos = {
    'Xception': Xception,
    'MobileNetV3Large': MobileNetV3Large
}
tolerancias = [0.005, 0.02]

# Función para construir el modelo
def build_model(base_model_fn, units_per_layer_list, activation='relu', learning_rate=0.001):
    base_model = base_model_fn(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.GlobalAveragePooling2D())
    for units in units_per_layer_list:
        model.add(layers.Dense(units, activation=activation))
        model.add(layers.Dropout(0.5))
    model.add(layers.Dense(y.shape[1], activation='softmax'))
    opt = optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Entrenamiento y evaluación
resultados = []

for nombre_modelo, modelo_base in modelos.items():
    for tolerancia in tolerancias:
        for num_layers, configs in layer_unit_configs.items():
            for units_list in configs:
                model = build_model(
                    base_model_fn=modelo_base,
                    units_per_layer_list=units_list,
                    activation=parametros_fijos['activation'],
                    learning_rate=parametros_fijos['learning_rate']
                )
                es = callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=parametros_fijos['patience'],
                    min_delta=tolerancia,
                    restore_best_weights=True
                )
                model.fit(
                    X_train, y_train,
                    epochs=parametros_fijos['epochs'],
                    batch_size=parametros_fijos['batch_size'],
                    validation_split=0.2,
                    callbacks=[es],
                    verbose=0
                )
                y_pred = model.predict(X_test)
                y_true = np.argmax(y_test, axis=1)
                y_pred_class = np.argmax(y_pred, axis=1)
                f1_macro = f1_score(y_true, y_pred_class, average='macro')
                f1_weighted = f1_score(y_true, y_pred_class, average='weighted')
                precision = precision_score(y_true, y_pred_class, average='macro', zero_division=0)
                recall = recall_score(y_true, y_pred_class, average='macro', zero_division=0)
                resultados.append({
                    'modelo': nombre_modelo,
                    'tolerancia': tolerancia,
                    'num_layers': num_layers,
                    'units_per_layer': str(units_list),
                    'f1_macro': f1_macro,
                    'f1_weighted': f1_weighted,
                    'precision': precision,
                    'recall': recall
                })

# Convertir a DataFrame y guardar
df_resultados = pd.DataFrame(resultados)
df_resultados.to_excel("/content/resultados_comparacion_modelos_tolerancia.xlsx", index=False)

# Resumen de la mejor combinación por modelo
top_config = df_resultados.sort_values(by='f1_macro', ascending=False).groupby('modelo').first().reset_index()
top_config.to_excel("/content/top_combinaciones_f1.xlsx", index=False)

# Gráficas comparativas por modelo
sns.set(style="whitegrid")

for metric in ['f1_macro', 'f1_weighted', 'precision', 'recall']:
    g = sns.catplot(
        data=df_resultados,
        kind='bar',
        x='num_layers',
        y=metric,
        hue='units_per_layer',
        col='modelo',
        height=6,
        aspect=1.2
    )
    g.fig.subplots_adjust(top=0.85)
    g.fig.suptitle(f'{metric.replace("_", " ").title()} por número de capas, estructura y tolerancia')
    g.set_axis_labels("Nº de capas", metric.replace("_", " ").title())
    g._legend.set_title("Estructura de neuronas")
    plt.savefig(f"/content/{metric}_comparacion_modelos_tolerancia.png")
    plt.show()

# Matriz de confusión para la mejor configuración por modelo
for idx, row in top_config.iterrows():
    modelo_nombre = row['modelo']
    mejor_modelo_fn = modelos[modelo_nombre]
    units = eval(row['units_per_layer'])
    model = build_model(
        base_model_fn=mejor_modelo_fn,
        units_per_layer_list=units,
        activation=parametros_fijos['activation'],
        learning_rate=parametros_fijos['learning_rate']
    )
    es = callbacks.EarlyStopping(
        monitor='val_loss',
        patience=parametros_fijos['patience'],
        min_delta=row['tolerancia'],
        restore_best_weights=True
    )
    model.fit(
        X_train, y_train,
        epochs=parametros_fijos['epochs'],
        batch_size=parametros_fijos['batch_size'],
        validation_split=0.2,
        callbacks=[es],
        verbose=0
    )
    y_pred = model.predict(X_test)
    y_pred_class = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y_test, axis=1)
    cm = confusion_matrix(y_true, y_pred_class)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap='Blues', xticklabels=clases_unicas, yticklabels=clases_unicas, cbar=True, square=True, linewidths=0)
    plt.title(f"Matriz de Confusión: {modelo_nombre}")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.tight_layout()
    plt.savefig(f"/content/matriz_confusion_{modelo_nombre}.png")
    plt.show()

# Exportar las mejores configuraciones detalladas
top_config.to_excel("/content/mejores_resultados_detalle.xlsx", index=False)
print("Exportadas mejores combinaciones detalladas a: mejores_resultados_detalle.xlsx")



